# Microbiota Sequence Processing
## Salmo salar mucosal tissue microbiota across functional feed treatments

#### Load Libraries

In [ ]:
set.seed(123)
library(dada2); packageVersion("dada2")
library(phyloseq); packageVersion("phyloseq")
library(ggplot2); packageVersion("ggplot2")
library(dplyr); packageVersion("dplyr")

#### Load raw fastq files

In [ ]:
path <- "splitFASTQ" 
head(list.files(path))

In [ ]:
fnFs <- sort(list.files(path, pattern="_R1.fastq", full.names = TRUE))
fnRs <- sort(list.files(path, pattern="_R2.fastq", full.names = TRUE))
sample.names <- sapply(strsplit(basename(fnFs), "_R1.fastq"), `[`, 1)
head(sample.names)

#### Plot read quality (F and R) for 4 random samples to determine trimming parameters

In [ ]:
randomNums <- sample(1:230, 4, replace=FALSE)
plotQualityProfile(fnFs[randomNums])
plotQualityProfile(fnRs[randomNums])

#### Run DADA2 workflow

In [ ]:
filtFs <- file.path(path, "filtered", paste0(sample.names, "_F_filt.fastq"))
filtRs <- file.path(path, "filtered", paste0(sample.names, "_R_filt.fastq"))

In [ ]:
out_PE <- filterAndTrim(fnFs, filtFs, fnRs, filtRs, truncLen=c(275,215), 
                        maxN=0, maxEE=c(2,3), truncQ=3, rm.phix=TRUE,
                        compress=TRUE, multithread=TRUE)

In [ ]:
head(out_PE)# truncLen=c(275,215) maxEE=c(2,3), truncQ=3
cat('Number of sequences after filtering:', sum(out_PE[,2]))

In [ ]:
errF <- learnErrors(filtFs, multithread=TRUE)
errR <- learnErrors(filtRs, multithread=TRUE)

In [ ]:
plotErrors(errF, nominalQ=TRUE)
plotErrors(errR, nominalQ=TRUE)

In [ ]:
derepFs <- derepFastq(filtFs)
derepRs <- derepFastq(filtRs)
names(derepFs) <- sample.names
names(derepRs) <- sample.names

In [ ]:
dadaFs <- dada(derepFs, err=errF, multithread=TRUE)
dadaRs <- dada(derepRs, err=errR, multithread=TRUE)

In [ ]:
mergers <- mergePairs(dadaFs, derepFs, dadaRs, derepRs)
head(mergers)

In [ ]:
seqtab <- seqtab[,nchar(colnames(seqtab)) > 400] #only merged ASVs greater than 400nt retained
dim(seqtab)[2]
table(nchar(getSequences(seqtab)))

In [ ]:
seqtab.nochim <- removeBimeraDenovo(seqtab, method="consensus", multithread=TRUE, verbose=TRUE)
sum(seqtab.nochim)/sum(seqtab)*100
dim(seqtab.nochim)[2]

In [ ]:
getN <- function(x) sum(getUniques(x))
track <- cbind(out_PE, sapply(dadaFs, getN), sapply(dadaRs, getN), sapply(mergers, getN), rowSums(seqtab.nochim))
colnames(track) <- c("input", "filtered", "denoisedF", "denoisedR", "merged", "noChimera")
rownames(track) <- sample.names
#head(track[order(track[,'merged']),], 50)

#### Assign taxonomy against SILVA 132

In [ ]:
taxa <- assignTaxonomy(seqtab.nochim, "/Users/jakebledsoe/Desktop/RBTVacSeqs/taxaReference/silva_nr_v132_train_set.fa", multithread=TRUE)

In [ ]:
saveRDS(taxa, file= 'GenusLevelTaxa.RDS')

In [ ]:
taxa <- addSpecies(taxa, "/Users/jakebledsoe/Desktop/RBTVacSeqs/taxaReference/silva_species_assignment_v132.fa")
saveRDS(taxa, file= 'SpeciesLevelTaxa.RDS')

In [ ]:
taxa <- readRDS('RDS_Data/SpeciesLevelTaxa.RDS')

#### Rearrange sample naming convention

In [ ]:
samples.out <- rownames(seqtab.nochim)
Tank <- sapply(strsplit(samples.out, "_"), `[`, 1)
Fish <- sapply(strsplit(samples.out, "_"), `[`, 2)
Tissue <- sapply(strsplit(samples.out, "_"), `[`, 3)
SampleID <- samples.out
meta.df <- data.frame(SampleID=SampleID, Tank=Tank, Fish=Fish, Tissue=Tissue)
rownames(meta.df) <- samples.out

In [ ]:
write.csv(meta.df, 'metaData.csv')

#### Construct phylogenetic tree for phylogenetically informed diversity metrics

In [ ]:
require('DECIPHER')
seqs <- getSequences(seqtab.nochim)
names(seqs) <- seqs 
alignment <- AlignSeqs(DNAStringSet(seqs), anchor=NA)

In [ ]:
library('phangorn')
fitGTR <- update(fit, k=4, inv=0.2)
fitGTR <- optim.pml(fitGTR, model="GTR", optInv=TRUE, optGamma=TRUE, rearrangement = "stochastic", control = pml.control(trace = 0))
detach('package:phangorn', unload=TRUE)
saveRDS(fitGTR, "phyfitGTR.RDS")

In [ ]:
fitGTR <- readRDS("RDS_Data/phyfitGTR.RDS")

In [ ]:
ps <- phyloseq(otu_table(seqtab.nochim, taxa_are_rows=FALSE), 
               sample_data(meta.df), 
               tax_table(taxa),
               phy_tree(fitGTR$tree))
ps

In [ ]:
dna <- Biostrings::DNAStringSet(taxa_names(ps))
names(dna) <- taxa_names(ps)
ps <- merge_phyloseq(ps, dna)
taxa_names(ps) <- paste0("ASV", seq(ntaxa(ps)))
ps

In [ ]:
saveRDS(ps, 'NCWMAC_ps_wTree.RDS')